In [1]:
import tensorflow as tf
import keras
from keras.layers import Dense, Input, Conv2D, 
from keras.models import Model
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

Using TensorFlow backend.
